## Step 1: Clone the PhysFormer repository and prepare the environment

In [1]:

!git clone 'https://github.com/ZitongYu/PhysFormer.git'
!wget 'https://drive.usercontent.google.com/download?id=1n1TpMQfU-OkZdJglEJyFp-vGo9JXbgsT&export=download&authuser=0&confirm=t&uuid=6e3d51ce-76c0-43cc-8799-db0a79976c3a&at=APZUnTW_BrrV1iI05BG41_sccbpk%3A1710947798275'
!mv '/content/download?id=1n1TpMQfU-OkZdJglEJyFp-vGo9JXbgsT&export=download&authuser=0&confirm=t&uuid=6e3d51ce-76c0-43cc-8799-db0a79976c3a&at=APZUnTW_BrrV1iI05BG41_sccbpk:1710947798275' 'v.zip'
!unzip '/content/v.zip'
!wget 'https://drive.usercontent.google.com/download?id=1jBSbM88fA-beaoVi8ILFyL0SvVVMA9c9&export=download&authuser=0&confirm=t&uuid=d1551966-5490-4665-a90e-eb60816659ac&at=APZUnTUt_KWH-tRaMFc_eaBw1Hdq%3A1710948205447'
!mv '/content/download?id=1jBSbM88fA-beaoVi8ILFyL0SvVVMA9c9&export=download&authuser=0&confirm=t&uuid=d1551966-5490-4665-a90e-eb60816659ac&at=APZUnTUt_KWH-tRaMFc_eaBw1Hdq:1710948205447' 'Physformer_VIPL_fold1.pkl'
!mv '/content/Physformer_VIPL_fold1.pkl' '/content/PhysFormer'
%cd '/content/PhysFormer'


Cloning into 'PhysFormer'...
remote: Enumerating objects: 69, done.
remote: Counting objects: 100% (62/62), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 69 (delta 24), reused 8 (delta 0), pack-reused 7 (from 1)
Unpacking objects: 100% (69/69), 5.67 MiB | 1.78 MiB/s, done.
--2024-08-15 18:13:29--  https://drive.usercontent.google.com/download?id=1n1TpMQfU-OkZdJglEJyFp-vGo9JXbgsT&export=download&authuser=0&confirm=t&uuid=6e3d51ce-76c0-43cc-8799-db0a79976c3a&at=APZUnTW_BrrV1iI05BG41_sccbpk%3A1710947798275
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.251.37.161, 2a00:1450:4006:811::2001
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|142.251.37.161|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 48611616 (46M) [application/octet-stream]
Saving to: ‘download?id=1n1TpMQfU-OkZdJglEJyFp-vGo9JXbgsT&export=download&authuser=0&confirm=t&uuid=6e3d51ce-76c0-43cc-8799-db0a79976c3a&at=APZUnTW_B

## Step 2: Load the required libraries and the pretrained model

In [6]:
# Clone the torchvision repository
!git clone https://github.com/pytorch/vision.git
!cd vision

# Install torchvision from source
!python setup.py install

Cloning into 'vision'...
remote: Enumerating objects: 560571, done.
remote: Counting objects: 100% (8357/8357), done.
remote: Compressing objects: 100% (564/564), done.
^Cceiving objects:  13% (72875/560571), 128.75 MiB | 2.63 MiB/s
/bin/bash: line 0: cd: vision: No such file or directory
/bin/bash: python: command not found


In [4]:
import cv2
import numpy as np
import torch
from torchvision import transforms
from model import ViT_ST_ST_Compact3_TDC_gra_sharp




AttributeError: partially initialized module 'torchvision' has no attribute 'extension' (most likely due to a circular import)

###  Initialize the model

In [ ]:
pytorch_model = ViT_ST_ST_Compact3_TDC_gra_sharp(image_size=(160, 128, 128), patches=(4, 4, 4), dim=96, ff_dim=144, num_heads=4, num_layers=12, dropout_rate=0.1, theta=0.7)
pytorch_model = pytorch_model.cuda()


###  Load pretrained weights

In [ ]:

pytorch_model.load_state_dict(torch.load('Physformer_VIPL_fold1.pkl'))  
pytorch_model.eval()

# Step 3: Prepare a sample video for inference

In [ ]:
def preprocess_video(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (128, 160))  
        frames.append(frame)
    cap.release()
    return np.array(frames)

# Example usage with a sample video file
sample_video = preprocess_video('path_to_sample_video.mp4')  # Provide the path to your sample video
sample_video_tensor = torch.from_numpy(sample_video).float().cuda()

# Run inference on the sample video
with torch.no_grad():
    output = pytorch_model(sample_video_tensor)